In [4]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl

from sklearn.ensemble import AdaBoostClassifier#adaboost引入方法
from sklearn.tree import DecisionTreeClassifier
from sklearn.datasets import make_gaussian_quantiles#造数据
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier,BaggingClassifier

In [5]:
## 设置属性防止中文乱码
mpl.rcParams['font.sans-serif'] = [u'SimHei']
mpl.rcParams['axes.unicode_minus'] = False

In [6]:
def entropy(t):
    return np.sum([-p*np.log2(p) for p in t if p!=0])

In [7]:
def h1(y=[1,1,1,-1,-1,-1,1,1,1,-1],w=[0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1]):
    #类似引用
    p1=np.sum(np.asarray(w)[np.asarray(y)==1])
    p2=1-p1
    return entropy([p1,p2])

In [8]:
# y=[1,1,1,-1,-1,-1,1,1,1,-1]
# w=[0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1]
# np.sum(np.asarray(w)[np.asarray(y)==1])

In [12]:
def h2(split=3,y=[1,1,1,-1,-1,-1,1,1,1,-1],w=[0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1]):
    y=np.asarray(y)
    w=np.asarray(w)
    #计算左边的相关信息
    data_y=y[:split]
    data_w=w[:split]
    p10=np.sum(data_w)
    p11=np.sum(data_w[data_y==1])/p10
    p12=1-p11
    h1=entropy([p11,p12])
    
    #计算右侧的相关信息
    data_y=y[split:]
    data_w=w[split:]
    p20=np.sum(data_w)
    p21=np.sum(data_w[data_y==1])/p20
    p22=1-p21
    h2=entropy([p21,p22])
    
    #计算划分的条件熵
    return p10*h1+p20*h2

def h3(errs=[],w=[0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1]):
    w=np.asarray(w)
    #计算错误率
    e=np.sum([w[i] for i in errs])
    #计算alpha
    alpha=0.5*np.log2((1-e)/e)
    #获取预测成功的样本下标
    trues=[i for i in range(len(w)) if i not in errs]
    #对预测成功的样本权重进行更新
    w[trues]=w[trues]*(np.e**(-alpha))
    #对预测失败的样本权重进行标注
    w[errs]=w[errs]*(np.e**(-alpha))
    # 做一个归一化，保证np.sum(w) == 1.0
    w = w / np.sum(w)

    return e,alpha,w
    
def calcl():
    print('第一个子模型的选择：')
    print('='*100)
    a=h1()
    print('原始数据的信息熵：{}'.format(a))
    print('以2.5划分的信息增益'.format(a-h2(split=3)))
    print('以5.5划分的信息增益'.format(a-h2(split=6)))
    print('以8.5划分的信息增益'.format(a-h2(split=9)))
    err,alpha1,w=h3(errs=[6,7,8])
    print("第1个子模型的误差率:{}".format(err))
    print("第1个子模型的权重系数:{}".format(alpha1))
    print("第1个子模型更新的样本权重系数:\n{}\n".format(w))
    
    
    print("第二个子模型的选择")
    print("=" * 100)
    a = h1(w=w)
    print("原始数据的信息熵:{}".format(a))
    print("以2.5划分的信息增益:{}".format(a - h2(split=3, w=w)))
    print("以5.5划分的信息增益:{}".format(a - h2(split=6, w=w)))
    print("以8.5划分的信息增益:{}".format(a - h2(split=9, w=w)))
    err, alpha2, w = h3(errs=[0, 1, 2, 9], w=w)
    print("第2个子模型的误差率:{}".format(err))
    print("第2个子模型的权重系数:{}".format(alpha2))
    print("第2个子模型更新的样本权重系数:\n{}\n".format(w))

    print("第三个子模型的选择")
    print("=" * 100)
    a = h1(w=w)
    print("原始数据的信息熵:{}".format(a))
    print("以2.5划分的信息增益:{}".format(a - h2(split=3, w=w)))
    print("以5.5划分的信息增益:{}".format(a - h2(split=6, w=w)))
    print("以8.5划分的信息增益:{}".format(a - h2(split=9, w=w)))
    err, alpha3, w = h3(errs=[3, 4, 5], w=w)
    print("第3个子模型的误差率:{}".format(err))
    print("第3个子模型的权重系数:{}".format(alpha3))
    print("第3个子模型更新的样本权重系数:\n{}\n".format(w))
    
    print("第四个子模型的选择")
    print("=" * 100)
    a = h1(w=w)
    print("原始数据的信息熵:{}".format(a))
    print("以2.5划分的信息增益:{}".format(a - h2(split=3, w=w)))
    print("以5.5划分的信息增益:{}".format(a - h2(split=6, w=w)))
    print("以8.5划分的信息增益:{}".format(a - h2(split=9, w=w)))
    err, alpha4, w = h3(errs=[6, 7, 8], w=w)
    print("第4个子模型的误差率:{}".format(err))
    print("第4个子模型的权重系数:{}".format(alpha4))
    print("第4个子模型更新的样本权重系数:\n{}\n".format(w))
    
    print("=" * 100)
    print("1:{}".format(alpha1 - alpha2 + alpha3 + alpha4))
    print("2:{}".format(-alpha1 - alpha2 + alpha3 - alpha4))
    print("3:{}".format(-alpha1 + alpha2 + alpha3 - alpha4))
    print("4:{}".format(-alpha1 + alpha2 - alpha3 - alpha4))
    
    
    
    
    
    

In [13]:
calcl()

第一个子模型的选择：
原始数据的信息熵：0.9709505944546686
以2.5划分的信息增益
以5.5划分的信息增益
以8.5划分的信息增益
第1个子模型的误差率:0.30000000000000004
第1个子模型的权重系数:0.6111962106682238
第1个子模型更新的样本权重系数:
[0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1]

第二个子模型的选择
原始数据的信息熵:0.9709505944546686
以2.5划分的信息增益:0.2812908992306925
以5.5划分的信息增益:0.046439344671015514
以8.5划分的信息增益:0.1444843438056279
第2个子模型的误差率:0.4
第2个子模型的权重系数:0.292481250360578
第2个子模型更新的样本权重系数:
[0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1]

第三个子模型的选择
原始数据的信息熵:0.9709505944546686
以2.5划分的信息增益:0.2812908992306925
以5.5划分的信息增益:0.046439344671015514
以8.5划分的信息增益:0.14448434380562802
第3个子模型的误差率:0.3
第3个子模型的权重系数:0.611196210668224
第3个子模型更新的样本权重系数:
[0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1]

第四个子模型的选择
原始数据的信息熵:0.9709505944546686
以2.5划分的信息增益:0.2812908992306925
以5.5划分的信息增益:0.046439344671015514
以8.5划分的信息增益:0.14448434380562802
第4个子模型的误差率:0.3
第4个子模型的权重系数:0.611196210668224
第4个子模型更新的样本权重系数:
[0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1]

1:1.5411073816440939
2:-0.9036774610288019
3:-0.31871496030764584
4:-1.5411073816440939

In [ ]:
# from sklearn import tree
# import pydotplus 
# dot_data = tree.export_graphviz(model, out_file=None) 
# graph = pydotplus.graph_from_dot_data(dot_data) 
# # graph.write_pdf("iris2.pdf") 
# graph.write_png("0616.png")